**For correct rendering, view this notebook in [nbviewer](https://nbviewer.org/github/markuskrecik/preference-dynamics-learning/blob/main/notebooks/30_feature_engineering.ipynb)**

# Feature Engineering

In the following, I use the `DataManager` capabilities to chain feature extractions for manually defined features.
Afterwards, I perform an automated feature extraction using tsfresh, visualize, and interpret the results.
To get an initial baseline for model performance, I set up simple linear regression models with the extracted features.

**This notebook:**
- Uses chained `DataTransformers` to perform manual feature engineering
- Automatically extracts features with `tsfresh`
- Visualizes and interprets feature correlations
- Sets up a baseline linear regression model with only manual features
- Compares the performance to a lr model including the tsfresh features

In [18]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import json
import numpy as np
import pandas as pd
import plotly.express as px
from plotly.offline import init_notebook_mode
from sklearn.linear_model import LinearRegression
import tsfresh as tsf

init_notebook_mode(connected=True)

from preference_dynamics.data import DataConfig, DataManager
from preference_dynamics.data.transformer import (
    DeleteSampleTransformer,
    InitialValueFeature,
    SteadyStateFeature,
    PeaksFeature,
    CyclesFeature,
    SampleGroupNormalizer,
)
from preference_dynamics.schemas import TimeSeriesSample
from preference_dynamics.training.metrics import compute_metrics
from preference_dynamics.visualization import plot_parameter_comparison, plot_metrics
from preference_dynamics.utils import get_param_names, get_var_names

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Manual Feature Engineering

First, I perform manual cleaning and feature engineering through these Transformers:
- `DeleteSampleTransformer` removes samples that were identified as divergent in the previous notebook.
- `InitialValueFeature` extracts the initial values as features.
- `SteadyStateFeature` identifies if the time series are in a steady state, and extracts the steady state value.
- `PeaksFeature` identifies peaks in the time series, as well as their distance and height for oscillatory time series.
- `CyclesFeature` identifies cycles (repeating patterns of peaks) in the time series and determines if the time series is in a limit cycle.

Finally, I normalize desires and efforts of each sample independently using the `SampleGroupNormalizer` Transformer, which also saves dataset statistics.


In [19]:
n_actions = 2
data_dir = f"data/n{n_actions}"

if n_actions == 1:
    remove_positions = []
elif n_actions == 2:
    remove_positions = [
        59,
        151,
        594,
        956,
        853,
        1194,
        2609,
        2834,
        3296,
        4607,
        6364,
        7749,
        7951,
        9890,
    ]

config = DataConfig(
    data_dir=data_dir,
    load_if_exists=False,
    transformers=[
        DeleteSampleTransformer(remove_positions=remove_positions),
        InitialValueFeature(),
        SteadyStateFeature(),
        PeaksFeature(),
        CyclesFeature(),
        SampleGroupNormalizer(),
    ],
)
dm = DataManager(config=config).setup()

In [20]:
# Feature selection


def expand_series_list(series: pd.Series, col_names: list[str]):
    return pd.DataFrame(series.tolist(), columns=col_names, index=series.index)


# features_keys = list(dm.splits["train"][0].features.keys())  # all features
features_keys = [
    "is_steady_state",
    "is_limit_cycle",
    "initial_value",
    "steady_state_mean",
    "limit_cycle_mean",
    "limit_cycle_mean_diff",
]

expand_cols = [
    "initial_value",
    "steady_state_mean",
    "limit_cycle_mean",
    "limit_cycle_mean_diff",
]

Y_names = get_param_names(n_actions, ic=True)

X_dfs = {}
Y_dfs = {}
for name, samples in dm.splits.items():
    # Create targets dataframes
    Y_data = [
        [s.parameters.values for s in samples],
        [s.initial_conditions.values for s in samples],
    ]
    Y_dfs[name] = pd.DataFrame(
        np.concatenate(Y_data, axis=1),
        columns=Y_names,
        index=[sample.sample_id for sample in samples],
    )

    # Select features
    features_data = [[sample.features[key] for key in features_keys] for sample in samples]
    X = pd.DataFrame(
        features_data,
        columns=features_keys,
        index=[sample.sample_id for sample in samples],
    )

    # Expand feature columns which contain lists
    for col in expand_cols:
        col_names = get_var_names(n_actions, col)
        expanded = expand_series_list(X[col], col_names)
        X = X.join(expanded)

    X.drop(columns=expand_cols, inplace=True)
    X.replace({np.nan: 0.0}, inplace=True)

    X_dfs[name] = X

display(X_dfs["train"].head())
display(Y_dfs["train"].head())

,is_steady_state,is_limit_cycle,u_0_initial_value,u_1_initial_value,a_0_initial_value,a_1_initial_value,u_0_steady_state_mean,u_1_steady_state_mean,a_0_steady_state_mean,a_1_steady_state_mean,u_0_limit_cycle_mean,u_1_limit_cycle_mean,a_0_limit_cycle_mean,a_1_limit_cycle_mean,u_0_limit_cycle_mean_diff,u_1_limit_cycle_mean_diff,a_0_limit_cycle_mean_diff,a_1_limit_cycle_mean_diff
dadc346a-7d37-4b63-be95-49fceea59411,True,False,-1.701802,3.823715,1.826466,0.260471,3.347190,-0.162178,1.844623,0.849967,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9f249f75-910d-4708-89b0-5a4d5d5c348d,True,False,-2.517420,2.969535,0.392103,0.000000,1.473820,0.096893,0.811141,1.457582,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5b39231a-e6ee-4a7e-b408-90618254783e,True,False,3.688731,1.116274,0.000000,1.594855,5.713349,13.956068,3.799532,3.082280,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4d8ffaba-55a3-40f2-8af5-b060b100fba4,True,False,3.459594,1.445251,1.205249,1.145456,0.581434,0.614538,0.182179,0.353060,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9578c8b0-0639-42fc-b705-63a6d55c6ac1,True,False,0.998460,-3.849617,1.547632,0.000000,-1.000599,2.707756,3.974104,4.108203,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,g_0,g_1,μ_0,μ_1,"Π_0,0","Π_0,1","Π_1,0","Π_1,1","Γ_0,0","Γ_0,1","Γ_1,0","Γ_1,1",v0_0,v0_1,m0_0,m0_1
dadc346a-7d37-4b63-be95-49fceea59411,2.601922,0.122892,-8.078731,-1.508125,1.943141,-1.155856,-1.311645,2.991154,2.672166,-1.861188,-0.407742,0.694090,-1.701802,3.823715,1.826466,0.260471
9f249f75-910d-4708-89b0-5a4d5d5c348d,1.779295,0.322713,-7.516077,-9.589566,2.321791,-0.071355,-2.202552,1.447118,1.367747,0.249992,-0.749352,0.483488,-2.517420,2.969535,0.392103,-1.724246
5b39231a-e6ee-4a7e-b408-90618254783e,2.412972,1.904047,-9.449673,-4.912609,2.808772,-2.679524,-1.470103,2.429938,2.017400,-0.633241,2.351993,1.628533,3.688731,1.116274,-0.748301,1.594855
4d8ffaba-55a3-40f2-8af5-b060b100fba4,0.226805,0.859190,-9.119071,-1.404714,1.814105,-0.293681,-0.934940,2.915984,1.578645,0.832263,0.457853,1.504355,3.459594,1.445251,1.205249,1.145456
9578c8b0-0639-42fc-b705-63a6d55c6ac1,0.723191,2.489131,-5.965116,-5.855241,2.618646,-2.357134,-2.431142,2.957679,1.667753,-1.856876,-2.005171,2.598828,0.998460,-3.849617,1.547632,-0.224380


## Automated Feature Engineering with tsfresh

tsfresh computes many predefined features, and selects relevant ones for time series to automate the feature engineering process.

In [21]:
# Compute tsfresh features, or load precomputed ones.
# Computation time ~25 min for n=2

load_features = False


def compute_and_save_tsfresh_features(
    samples: list[TimeSeriesSample], n_actions: int, split_name: str, settings: dict | None = None
):
    dfs = []
    for sample in samples:
        col_names = get_var_names(n_actions)
        sample_df = pd.DataFrame(sample.time_series.T, columns=col_names)
        sample_df["time"] = sample.time_points
        sample_df["id"] = sample.sample_id
        dfs.append(sample_df)
    df = pd.concat(dfs)

    features = tsf.extract_features(
        df,
        column_id="id",
        column_sort="time",
        column_kind=None,
        column_value=None,
        kind_to_fc_parameters=settings,
    )

    tsf.utilities.dataframe_functions.impute(features)  # Remove or impute all missing features

    features.to_csv(Path(data_dir) / f"processed/tsfresh_features_{split_name}.csv")

    return features


X_tsfresh_dfs = {}
for split_name, samples in dm.splits.items():
    if load_features:
        X_tsfresh_dfs[split_name] = pd.read_csv(
            Path(data_dir) / f"processed/tsfresh_features_{split_name}.csv", index_col=0
        )
    else:
        try:
            with open(Path(data_dir) / "processed/tsfresh_features_settings.json", "r") as f:
                tsfresh_settings = json.load(f)
        except FileNotFoundError:
            tsfresh_settings = None

        X_tsfresh_dfs[split_name] = compute_and_save_tsfresh_features(
            samples, n_actions, split_name, tsfresh_settings
        )

X_tsfresh_dfs["train"].head()

Feature Extraction: 100%|██████████| 30/30 [15:55<00:00, 31.85s/it]  
/home/igor/git/preference-dynamics-learning/.venv/lib/python3.13/site-packages/tsfresh/utilities/dataframe_functions.py:198: RuntimeWarning:

The columns ['u_0__query_similarity_count__query_None__threshold_0.0'
 'u_1__query_similarity_count__query_None__threshold_0.0'
 'a_0__query_similarity_count__query_None__threshold_0.0'
 'a_1__query_similarity_count__query_None__threshold_0.0'] did not have any finite values. Filling with zeros.

Feature Extraction: 100%|██████████| 30/30 [02:42<00:00,  5.41s/it]
/home/igor/git/preference-dynamics-learning/.venv/lib/python3.13/site-packages/tsfresh/utilities/dataframe_functions.py:198: RuntimeWarning:

The columns ['u_0__query_similarity_count__query_None__threshold_0.0'
 'u_1__query_similarity_count__query_None__threshold_0.0'
 'a_0__query_similarity_count__query_None__threshold_0.0'
 'a_1__query_similarity_count__query_None__threshold_0.0'] did not have any finite values. Fil

,u_0__variance_larger_than_standard_deviation,u_0__has_duplicate_max,u_0__has_duplicate_min,u_0__has_duplicate,u_0__sum_values,u_0__abs_energy,u_0__mean_abs_change,u_0__mean_change,u_0__mean_second_derivative_central,u_0__median,...,a_1__fourier_entropy__bins_5,a_1__fourier_entropy__bins_10,a_1__fourier_entropy__bins_100,a_1__permutation_entropy__dimension_3__tau_1,a_1__permutation_entropy__dimension_4__tau_1,a_1__permutation_entropy__dimension_5__tau_1,a_1__permutation_entropy__dimension_6__tau_1,a_1__permutation_entropy__dimension_7__tau_1,a_1__query_similarity_count__query_None__threshold_0.0,a_1__mean_n_absolute_max__number_of_maxima_7
00040355-424f-43be-9973-9d3fa7b7821e,0.0,0.0,0.0,0.0,-84.067474,114.503798,0.807966,0.001155,-0.000060,-0.439967,...,0.110993,0.166340,0.331771,1.689175,2.332656,2.721796,3.036370,3.308429,0.0,2.248472
000eaaa1-637c-4553-b5fa-73a5018aeff4,0.0,0.0,0.0,0.0,146.930805,224.435574,0.026156,0.026155,-0.001537,0.960153,...,0.110993,0.110993,0.110993,1.384031,2.283048,2.988825,3.238239,3.336546,0.0,1.432256
000f5c98-40bc-4bf7-89d4-fbfb4666e7b1,0.0,0.0,0.0,0.0,-17.858403,190.518149,0.025483,0.025483,-0.001347,0.345395,...,0.097267,0.110993,0.166340,1.167835,1.786173,2.122286,2.380864,2.531696,0.0,1.167071
000f880c-1429-4a24-ac25-95fc9213b187,0.0,0.0,0.0,0.0,-197.194283,196.659635,0.022906,0.007866,-0.003447,-0.973486,...,0.110993,0.110993,0.110993,1.713849,2.570561,3.355873,4.044825,4.541598,0.0,1.393027
00126c3c-be26-41f3-8be2-15ca2522d89a,0.0,0.0,0.0,0.0,-197.656038,197.855996,0.008313,-0.004056,-0.000451,-1.001134,...,0.110993,0.110993,0.110993,1.669369,2.758133,3.847216,4.350578,4.673806,0.0,2.141780


In [22]:
# Select relevant features, and compute their correlation with parameters.
# Only compute pairwise correlations with params and IC, as the whole corr matrix is too large.
# Use spearman, because we're not only interested in the linear relationships
# Time ~4 min for n=2

relevant_features = {}
corr = {}
for param in Y_names:
    y = Y_dfs["train"][param].sort_index()
    relevant_features[param] = tsf.select_features(X_tsfresh_dfs["train"], y)
    corr[param] = relevant_features[param].corrwith(y, method="spearman")

In [23]:
# Save relevant features settings, so we only recompute these
tsfresh_settings = {}
for param in Y_names:
    tsfresh_settings.update(tsf.feature_extraction.settings.from_columns(relevant_features[param]))

with open(Path(data_dir) / "processed/tsfresh_features_settings.json", "w") as f:
    json.dump(tsfresh_settings, f)

## Visualize correlations

### Manual Feature Engineering

The manually computed features are moderately to weakly correlated with the parameters.
The initial values are highly correlated with the initial conditions, which is expected, since they are the same as the initial conditions for $\mathbf{v}>\mathbf{\mu}$ and $\mathbf{m}>0$.
$\mathbf{\mu}$ is almost not correlated at all, which hints at an identifiability problem. This comes as no surprise, since it only affects the dynamics for $\mathbf{\mu}>0$, which I however excluded from the parameter space for now.


In [24]:
corr_features = X_dfs["train"].join(Y_dfs["train"]).corr(method="spearman")
n_features = len(corr_features)

fig = px.imshow(
    corr_features,
    color_continuous_scale="RdBu_r",
    zmin=-1,
    zmax=1,
    height=n_features * 20 + 200,
)
fig.update_xaxes(side="top")
fig.show()

### tsfresh Features

Not all features labeled relevant by tsfresh are highly correlated with the parameters and IC.
We only keep the most promising ones and visualize their correlations.

The aggregated feature correlations with the parameters (sum over abs(corr)) gives a first indication of how well the parameters can be predicted from the features. The desire threshold $\mathbf{\mu}$, and goals $\mathbf{g}$ will be hardest to predict from these features.

The heatmap shows the pairwise correlations of the features. In future steps, highly correlated features have to be trimmed down, and the interpretation of each feature has to be analyzed in detail.

In [25]:
threshold = 0.25
corr_important = {param: c[c.abs() > threshold] for param, c in corr.items()}
corr_important_df = pd.DataFrame(corr_important)
n_features = len(corr_important_df)

print("Aggregated feature importance per parameter:")
display(corr_important_df.abs().sum().sort_values(ascending=False))

fig = px.imshow(
    corr_important_df,
    title=f"Correlation of {n_features} tsfresh Features with Parameters",
    color_continuous_scale="RdBu_r",
    zmin=-1,
    zmax=1,
    height=n_features * 20 + 200,
)
fig.update_xaxes(side="top")
fig.show()

Aggregated feature importance per parameter:


Γ_0,1    167.774197
Γ_1,0    123.735499
v0_0     117.652538
v0_1     112.239123
Π_0,1     45.598802
Π_1,1     44.282013
Π_1,0     42.354821
Γ_1,1     37.120076
Π_0,0     37.073627
m0_1      26.169882
m0_0      24.953420
Γ_0,0     24.231248
μ_0        1.993795
μ_1        1.990919
g_1        1.947300
g_0        0.851473
dtype: float64

Having seen the overall feature importance above, it is also interesting to know if any of the time series are particularly good in predicting the parameters.
In particular, I want to know if either desires (u) or efforts (a) are more informative; and which action (0 or 1) is informative for which parameter. Again, I sum over the abs correlation coefficients.

The first plot shows that desire time series features have overall more predictive power. In other words, if desires are unobservable (as is the case in the real world), prediction will be relatively harder.
Furthermore, $\mathbf{u}(t)$ predicts $\Gamma$, $\mathbf{\mu}$, and $\mathbf{v}_0$ better; while $\mathbf{a}(t)$ predicts $\Pi$, $\mathbf{g}$, and $\mathbf{m}_0$ better (and interestingly, the opposite parameters to the ODE specification, see the first notebook).

The second plot shows that each action is better in predicting its "own" parameters (i.e., with the same index). Therefore, if only action 0 is observed, it will be harder to predict the parameters of action 1.

For completeness, the last plot shows the role of each individual time series, which confirms the previous findings.


In [26]:
var_names_df = pd.Series(corr_important_df.index, index=corr_important_df.index).str.split(
    "_", n=2, expand=True
)
var_names_df.columns = ["var_name", "var_index", "var"]
var_names_df["var"] = var_names_df["var_name"] + "_" + var_names_df["var_index"]

var_corr_df = var_names_df.join(corr_important_df.abs())

name_param_importance = var_corr_df.groupby("var_name").sum(numeric_only=True)
index_param_importance = var_corr_df.groupby("var_index").sum(numeric_only=True)
index_param_importance = index_param_importance[
    index_param_importance.columns.sort_values(key=lambda col: col.str.split("_").str[-1])
]
var_param_importance = var_corr_df.groupby("var").sum(numeric_only=True)
var_param_importance = var_param_importance[
    var_param_importance.columns.sort_values(key=lambda col: col.str.split("_").str[-1])
]
var_param_importance = var_param_importance.sort_index(
    key=lambda idx: idx.str.split("_").str[-1], ascending=False
)


px.imshow(
    name_param_importance,
    title="Importance of efforts a vs desires u in parameter estimation",
    labels={"x": "Parameter", "y": "Desire u, effort a", "color": "Importance"},
    color_continuous_scale="Reds",
    zmax=40,
).show()

px.imshow(
    index_param_importance,
    title="Importance of action (0 or 1) in parameter estimation",
    labels={"x": "Parameter", "y": "Action", "color": "Importance"},
    color_continuous_scale="Reds",
    zmax=40,
).show()

px.imshow(
    var_param_importance,
    title="Importance of each time series in parameter estimation",
    labels={"x": "Parameter", "y": "Action Desires u, Effort a", "color": "Importance"},
    color_continuous_scale="Reds",
    zmax=40,
).show()

## Baseline model: Linear Regression

Let's do a quick baseline test with linear regression. I compare the model with only manual features to the full model including all tsfresh features. Since I only compare predictions, I don't have to worry about near-collinearity.

### Minimal Model: Only Manual Features


In [27]:
# Manual feature set
X_train = X_dfs["train"]
X_test = X_dfs["test"]

Y_train = Y_dfs["train"]
Y_test = Y_dfs["test"]

lr = LinearRegression()
lr.fit(X_train, Y_train)

Y_pred = lr.predict(X_test)

In [28]:
# Visualize performance and metrics
target_names = get_param_names(n_actions, ic=True)
metrics = compute_metrics(Y_test, Y_pred)
plot_metrics(metrics, target_names=target_names, width=1000, height=1000)

plot_parameter_comparison(Y_test, Y_pred, col_names=Y_names, width=1000, height=1000)

The minimal model is generally bad at predictions, as e.g. exemplified by the MRE>2 (The model is overall 200% wrong across all parameters).
It fares well for the initial conditions (if they are above their thresholds).
$\mathbf{\mu}$, $\mathbf{g}$, and the off-diagonal elements of $\Pi$ are particularly hard to predict, which will be a repeating pattern.

### Full Model: All Features

In [29]:
# Full feature set
X_train = X_dfs["train"].join(X_tsfresh_dfs["train"])
X_test = X_dfs["test"].join(X_tsfresh_dfs["test"])

Y_train = Y_dfs["train"]
Y_test = Y_dfs["test"]

lr_full = LinearRegression()
lr_full.fit(X_train, Y_train)

Y_pred_full = lr_full.predict(X_test)

In [30]:
# Visualize performance and metrics
target_names = get_param_names(n_actions, ic=True)
metrics_full = compute_metrics(Y_test, Y_pred_full)
plot_metrics(metrics_full, target_names=target_names, width=1000, height=1000)

plot_parameter_comparison(Y_test, Y_pred_full, col_names=Y_names, width=1000, height=1000)

The full model fares slightly better than the minimal model. It starts to reproduce linear relationships in the parameters, captured by the high correlation, but at the cost of predicting the initial conditions.
The lower R² however indicates a large bias.
Furthermore, RMSE > MAE > Median absolute error (MedAE) shows us that some predictions of the linear model are large outliers.

### Comparison of Models

In [31]:
# Difference in metrics to minimal model
metrics_diff = {k: metrics_full[k] - v for k, v in metrics.items()}
plot_metrics(
    metrics_diff,
    target_names=target_names,
    width=1000,
    height=1200,
    title="Difference in metrics to minimal model",
)

The metrics differences to the minimal model show that the full model captures each parameter slightly better, 
but at the cost of vastly worsening on the initial conditions.
Thus, almost all metrics of the full model are worse than the minimal model.

## Summary

The feature engineering reveals:
- All extracted features are only weakly to moderately correlated with the parameters, making the use of simple models challenging.
- Desire time series have more predictive power than efforts.
- Each action is better in predicting its own parameters.
- The inverse problem is ill-posed, making the prediction of $\mathbf{\mu}$ and $\mathbf{g}$ challenging, which is confirmed through simple linear regression models.

**Future extensions:**
- Extend manual feature engineering with: Time series min/max amplitudes, limit cycle min/max, wavelet spectrum
- Trim down highly correlated features

**Next steps** (see `40_training_cnn_n1.ipynb`):
- Train a 1d CNN model for the easiest case of n=1 action
- Evaluate its performance